# The Canonical Time-series Characteristics (catch22) transform

catch22\[1\] is a collection of 22 time series features extracted from the 7000+ present in the _hctsa_\[2\]\[3\] toolbox.
A hierarchical clustering was performed on the correlation matrix of features that performed better than random chance to remove redundancy.
These clusters were sorted by balanced accuracy using a decision tree classifier and a single feature was selected from the 22 clusters formed, taking into account balanced accuracy results, computational efficiency and interpretability.

In this notebook, we will demonstrate how to use the catch22 transformer on the ItalyPowerDemand univariate and BasicMotions multivariate datasets. We also show catch22 used for classication with a random forest classifier.

Both make use of the features implemented in the catch22 package (<https://github.com/chlubba/catch22>), where versions of catch22 for C and MATLAB are also available.

<span style="color:red">**The catch22 package is currently unstable, results will differ between operating systems.**</span>

#### References:

\[1\] Lubba, C. H., Sethi, S. S., Knaute, P., Schultz, S. R., Fulcher, B. D., & Jones, N. S. (2019). catch22: CAnonical Time-series CHaracteristics. Data Mining and Knowledge Discovery, 33(6), 1821-1852.

\[2\] Fulcher, B. D., & Jones, N. S. (2017). hctsa: A computational framework for automated time-series phenotyping using massive feature extraction. Cell systems, 5(5), 527-531.

\[3\] Fulcher, B. D., Little, M. A., & Jones, N. S. (2013). Highly comparative time-series analysis: the empirical structure of time series and their methods. Journal of the Royal Society Interface, 10(83), 20130048.

## 1. Imports

In [1]:
from sklearn import metrics

from sktime.contrib.hybrid._catch22_forest_classifier import Catch22ForestClassifier
from sktime.contrib.transformers.catch22_features import Catch22
from sktime.datasets import load_basic_motions, load_italy_power_demand

## 2. Load data

In [2]:
IPD_X_train, IPD_y_train = load_italy_power_demand(split="train", return_X_y=True)
IPD_X_test, IPD_y_test = load_italy_power_demand(split="test", return_X_y=True)
IPD_X_test = IPD_X_test[:50]
IPD_y_test = IPD_y_test[:50]

print(IPD_X_train.shape, IPD_y_train.shape, IPD_X_test.shape, IPD_y_test.shape)

BM_X_train, BM_y_train = load_basic_motions(split="train", return_X_y=True)
BM_X_test, BM_y_test = load_basic_motions(split="test", return_X_y=True)

print(BM_X_train.shape, BM_y_train.shape, BM_X_test.shape, BM_y_test.shape)

(67, 1) (67,) (1029, 1) (1029,)
(40, 6) (40,) (40, 6) (40,)


## 3. catch22 transform

### Univariate

The catch22 features are provided in the form of a transformer, `Catch22`.
From this the transformed data can be used for a variety of time series analysis tasks.

In [3]:
c22_uv = Catch22()
c22_uv.fit(IPD_X_train, IPD_y_train)

Catch22()

In [4]:
transformed_data_uv = c22_uv.transform(IPD_X_train)
print(transformed_data_uv.head())

         0         1   2   3         4         5         6     7         8   \
0  1.158650 -0.217228   3   6  0.485847  0.095361  1.000000   8.0  0.040000   
1  0.918174 -0.214746   4   8  0.548270  0.071349  0.869565  15.0  0.111111   
2 -0.273186 -0.085866   2   5  0.464109  0.216576  0.913043   3.0  0.034014   
3  0.048404 -0.450092   4  10  0.609319  0.124926  0.869565  13.0  0.111111   
4  0.426386 -0.450726   4   7  0.559022  0.054549  0.913043  16.0  0.111111   

   9   ...        12        13        14        15   16        17   18   19  \
0   0  ...  0.750000  0.291667 -0.625000  0.468050  5.0  1.787502  0.0  0.0   
1   0  ...  0.500000  0.208333 -0.666667  0.702777  5.0  1.730238  0.0  0.0   
2   0  ...  0.666667  0.875000  0.250000  0.310570  5.0  1.730238  0.0  0.0   
3   0  ...  0.666667  0.166667 -0.625000  0.804046  6.0  1.605420  0.0  0.0   
4   0  ...  0.500000  0.291667 -0.666667  0.675482  6.0  1.730238  0.0  0.0   

         20        21  
0  0.589049  0.857423  
1 

The transform `Catch22` method will process all 22 features.
For individual features, the transform_single_feature method can be used when provided with a numeric feature ID or the feature name.

In [5]:
transformed_feature_uv = c22_uv._transform_single_feature(IPD_X_train, "CO_f1ecac")
print(transformed_feature_uv)

[3 4 2 4 4 3 3 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 3 4 4 3 4 4
 3 4 4 3 4 4 3 4 4 3 4 3 3 4 3 4 3 3 4 4 4 3 2 4 4 4 4 4 4 2]


### Multivariate

Transformation of multivariate data is supported by `Catch22`.
The default procedure will concatenate each column prior to transformation.

In [6]:
c22_mv = Catch22()
c22_mv.fit(BM_X_train, BM_y_train)

Catch22()

In [7]:
transformed_data_mv = c22_mv.transform(BM_X_train)
print(transformed_data_mv.head())

         0         1   2   3         4         5         6     7         8   \
0 -0.417887  0.187460   2   5  0.105551 -0.649750  0.794658   9.0  0.002256   
1  1.061005  0.294088   2   6  0.043438 -1.968673  0.626043  79.0  0.012387   
2 -0.065333  0.297270   3   6  0.111391 -0.015163  0.808013  25.0  0.000808   
3  0.274125 -0.130168   2   5  0.062411 -0.103788  0.841402  17.0  0.005236   
4 -0.310012  0.068879   3   5  0.086561  0.321862  0.813022  26.0  0.012837   

   9   ...        12        13        14        15   16        17        18  \
0   9  ...  0.666667 -0.576667 -0.421667  0.479256  7.0  1.952422  0.127660   
1  10  ...  0.250000 -0.501667 -0.318333  0.603359  7.0  1.858507  0.170213   
2  11  ...  1.000000 -0.446667 -0.470000  0.812749  8.0  1.927062  0.127660   
3   8  ...  0.666667 -0.366667 -0.351667  0.486981  7.0  2.009502  0.127660   
4  10  ...  0.750000 -0.501667 -0.695833  0.767016  7.0  1.869634  0.787234   

         19        20        21  
0  0.659574  0.6

## 4. catch22 Forest Classifier

For classification tasks the default classifier to use with the catch22 features is random forest classifier.
An implementation making use of the `RandomForestClassifier` from sklearn built on catch22 features is provided in the form on the `Catch22ForestClassifier` for ease of use.

In [8]:
c22f = Catch22ForestClassifier(n_estimators=100, random_state=0)
c22f.fit(IPD_X_train, IPD_y_train)

Catch22ForestClassifier(random_state=0)

In [9]:
c22f_preds = c22f.predict(IPD_X_test)
print("C22F Accuracy: " + str(metrics.accuracy_score(IPD_y_test, c22f_preds)))

C22F Accuracy: 0.8794946550048591
